In [1]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [13]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision.models import ResNet

from source.models import BasicBlock
from source.data import get_training_dataloader, get_test_dataloader

In [6]:
CIFAR100_TRAIN_MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
CIFAR100_TRAIN_STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
MILESTONES = [60, 120, 160]
# data_path = '/gpfs/gpfs0/k.sobolev/cifar-100-python/cifar-100-python/'
data_path = '../data'

In [19]:
cifar100_training_loader = get_training_dataloader(
    data_path,
    CIFAR100_TRAIN_MEAN,
    CIFAR100_TRAIN_STD,
    num_workers=4,
    batch_size=64,
    shuffle=True
)

cifar100_test_loader = get_test_dataloader(
    data_path,
    CIFAR100_TRAIN_MEAN,
    CIFAR100_TRAIN_STD,
    num_workers=4,
    batch_size=64,
    shuffle=True
)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
model = ResNet(num_classes=100, block=BasicBlock, layers=[2, 2, 2, 2])
model.eval()
model = model.cuda()

In [15]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

In [17]:
best_acc = 0.0

for epoch in range(10):
    model.train()
    for batch_index, (images, labels) in enumerate(cifar100_training_loader):
        labels = labels.cuda()
        images = images.cuda()
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        test_loss = 0.0
        correct = 0.0
        
        for (images, labels) in cifar100_test_loader:
            images = images.cuda()
            labels = labels.cuda()

            outputs = model(images)
            loss = loss_function(outputs, labels)

            test_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum()
    
    val_acc = correct.float() / len(cifar100_test_loader.dataset)
    print('Test set: Epoch: {}, Average loss: {:.4f}, Accuracy: {:.4f}'.format(
        epoch,
        test_loss / len(cifar100_test_loader.dataset),
        val_acc,
    ))
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), '../models/resnet18_cifar100.sd')


Test set: Epoch: 0, Average loss: 0.0543, Accuracy: 0.1604
Test set: Epoch: 1, Average loss: 0.0507, Accuracy: 0.2040
Test set: Epoch: 2, Average loss: 0.0477, Accuracy: 0.2395
Test set: Epoch: 3, Average loss: 0.0454, Accuracy: 0.2765
Test set: Epoch: 4, Average loss: 0.0462, Accuracy: 0.2664
Test set: Epoch: 5, Average loss: 0.0442, Accuracy: 0.2954
Test set: Epoch: 6, Average loss: 0.0448, Accuracy: 0.2922
Test set: Epoch: 7, Average loss: 0.0417, Accuracy: 0.3220
Test set: Epoch: 8, Average loss: 0.0416, Accuracy: 0.3212
Test set: Epoch: 9, Average loss: 0.0437, Accuracy: 0.3153


In [ ]:
# loss_function = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
# train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=0.2) #learning rate decay

In [ ]:
# for epoch in range(10):
#     model.train()
#     for batch_index, (images, labels) in enumerate(cifar100_training_loader):
#         labels = labels.cuda()
#         images = images.cuda()
        
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = loss_function(outputs, labels)
#         loss.backward()
#         optimizer.step()
    
#     model.eval()
#     with torch.no_grad():
#         test_loss = 0.0
#         correct = 0.0
        
#         for (images, labels) in cifar100_test_loader:
#             images = images.cuda()
#             labels = labels.cuda()

#             outputs = model(images)
#             loss = loss_function(outputs, labels)

#             test_loss += loss.item()
#             _, preds = outputs.max(1)
#             correct += preds.eq(labels).sum()
    
#     print('Test set: Epoch: {}, Average loss: {:.4f}, Accuracy: {:.4f}, Lr:{:.4f}'.format(
#         epoch,
#         test_loss / len(cifar100_test_loader.dataset),
#         correct.float() / len(cifar100_test_loader.dataset),
#         scheduler.get_last_lr()[0]
#     ))
        
#     train_scheduler.step(epoch)